# A*Star BII programming interview

### Question 1

#### Single path

In [6]:
class question_1:
    def run(self, input_file, output_file):
        lines = self.read_input(input_file)
        output_string = self.generate_all_outputs(lines)
        self.write_output(output_string, output_file)
    
    def read_input(self, input_file):
        with open(input_file, 'r') as rf:
            lines = rf.readlines()
        return lines
    
    def generate_all_outputs(self, lines):
        output_string = ''
        for line in lines:
            line = line.replace('\n', '')
            if len(line):
                output_string += self.generate_one_output(line)
            output_string += '\n'
        return output_string
    
    def generate_one_output(self, line):
        m, n, sum_num = self.parse_inputs(line)
        walker = self.grid_walker(m, n, sum_num)
        if not walker.check_input_validity():
            return ''
        history = ''
        while not walker.at_end():
            operation = walker.move()
            history += operation
        return history
    
    def parse_inputs(self, line):
        m, n, sum_num = line.split(' ')
        return int(m), int(n), int(sum_num)
    
    def write_output(self, output_string, output_file):
        with open(output_file, 'w') as wf:
            wf.write(output_string)
    
    class grid_walker:
        def __init__(self, m, n, sum_num):
            self.x = 1
            self.y = 1
            self.cum_sum = 1
            self.n = n
            self.m = m
            self.sum_num = sum_num
        
        def compute_minimum(self, x, y):
            return y*(self.n-x) + sum(range(y, self.m+1))

        def compute_maximum(self, x, y):
            return self.n*(self.n-x) + sum(range(y, self.m+1))
        
        def check_input_validity(self):
            min_sum = self.compute_minimum(self.x, self.y)
            max_sum = self.compute_maximum(self.x, self.y)
            is_valid = self.sum_num >= min_sum and self.sum_num <= max_sum
            if not is_valid:
                print('sum:', self.sum_num, 'is invalid')
                print('min:', min_sum)
                print('max:', max_sum)
                if self.sum_num > max_sum:
                    print('Sum exceeds maximum sum possible using this grid!\nTry a value within [', 
                          min_sum, ',', max_sum, ']', sep='')
                else:
                    print('Sum is too small; unable to reach the end of this grid!\nTry a value within [', 
                          min_sum, ',', max_sum, ']', sep='')
            return is_valid
        
        def update_state(self, operation):
            if operation == 'R':
                self.x += 1
            elif operation == 'D':
                self.y += 1
            self.cum_sum += self.y
        
        def at_right_edge(self):
            return self.x == self.n
        
        def at_bottom_edge(self):
            return self.y == self.m
        
        def at_end(self):
            return self.at_right_edge() and self.at_bottom_edge()
        
        def down_is_valid(self):
            return (self.compute_minimum(self.x, self.y+1) + self.cum_sum) <= self.sum_num and \
                   not self.at_bottom_edge()
        
        def move(self):
            if self.down_is_valid():
                operation = 'D'
            else:
                operation = 'R'
            self.update_state(operation)
            return operation

In [8]:
qn = question_1()
qn.run('input_question_1', 'output_question_1')

sum: 87127231192 is invalid
min: 5000139999
max: 13099960000
Sum exceeds maximum sum possible using this grid!
Try a value within [5000139999,13099960000]


A grid with m rows and n columns has a max sum of $(n-1)*m+\frac{m}{2}*(m+1)$.

In this case, the max sum is $(100,000-1)*90,000+\frac{90,000}{2}*(90,000+1)=13,049,955,000$.

#### All paths

We will only run this on the two smaller problems (4x4 and 9x9 grids). We won't write an output file since this is not part on the question.

In [94]:
class question_1:
    def run(self, input_file):
        lines = self.read_input(input_file)
        paths = self.generate_all_outputs(lines)
        self.print_results(paths, lines)
    
    def read_input(self, input_file):
        with open(input_file, 'r') as rf:
            lines = rf.readlines()
        return lines
    
    def generate_all_outputs(self, lines):
        paths = []
        for line in lines:
            line = line.replace('\n', '')
            if len(line):
                paths.append(self.generate_one_output(line))
        return paths
    
    def generate_one_output(self, line):
        m, n, sum_num = self.parse_inputs(line)
        if m > 10 or n > 10:
            return ''
        walker = self.grid_walker(m, n, sum_num)
        if not walker.check_input_validity():
            return ''
        histories = []
        while not walker.stack_is_empty():
            walker.check_next_path()
            while not walker.done():
                operation = walker.move()
            if not walker.invalid_position():
                histories.append(walker.get_history())
        return histories
    
    def parse_inputs(self, line):
        m, n, sum_num = line.split(' ')
        return int(m), int(n), int(sum_num)
    
    def print_results(self, paths, lines):
        lines = [line.replace('\n', '') for line in lines]
        lines = [line for line in lines if line]
        for i in range(len(paths)):
            if paths[i]:
                print('input:', lines[i])
                print('paths found:', len(paths[i]))
                if len(paths[i]) < 500:
                    print(paths[i])
                print('')
    
    class grid_walker:
        def __init__(self, m, n, sum_num):
            self.x = 1
            self.y = 1
            self.cum_sum = 1
            self.n = n
            self.m = m
            self.sum_num = sum_num
            self.stack = [((1, 1), '', 1)]
            self.history = ''
        
        def compute_minimum(self, x, y):
            return y*(self.n-x) + sum(range(y, self.m+1))

        def compute_maximum(self, x, y):
            return self.n*(self.n-x) + sum(range(y, self.m+1))
        
        def check_input_validity(self):
            min_sum = self.compute_minimum(self.x, self.y)
            max_sum = self.compute_maximum(self.x, self.y)
            is_valid = self.sum_num >= min_sum and self.sum_num <= max_sum
            if not is_valid:
                print('sum:', self.sum_num, 'is invalid')
                print('min:', min_sum)
                print('max:', max_sum)
                if self.sum_num > max_sum:
                    print('Sum exceeds maximum sum possible using this grid!\nTry a value within [', 
                          min_sum, ',', max_sum, ']', sep='')
                else:
                    print('Sum is too small; unable to reach the end of this grid!\nTry a value within [', 
                          min_sum, ',', max_sum, ']', sep='')
            return is_valid
        
        def update_state(self, operation):
            if operation == 'R':
                self.x += 1
            elif operation == 'D':
                self.y += 1
            self.cum_sum += self.y
            self.history += operation
        
        def at_right_edge(self):
            return self.x >= self.n
        
        def at_bottom_edge(self):
            return self.y >= self.m
        
        def at_end(self):
            return self.at_right_edge() and self.at_bottom_edge() and self.cum_sum == self.sum_num
        
        def down_is_valid(self):
            return (self.compute_minimum(self.x, self.y+1) + self.cum_sum) <= self.sum_num and \
                   not self.at_bottom_edge()
        
        def move(self):
            if self.down_is_valid():
                operation = 'D'
                if not self.at_right_edge():
                    self.stack.append(((self.x+1, self.y), self.history+'R', self.cum_sum+self.y))
            else:
                operation = 'R'
            self.update_state(operation)
            return operation
        
        def stack_is_empty(self):
            return not len(self.stack)
        
        def check_next_path(self):
            last_possible = self.stack.pop()
            self.x, self.y = last_possible[0]
            self.history = last_possible[1]
            self.cum_sum = last_possible[2]
        
        def get_history(self):
            return self.history
        
        def done(self):
            return self.at_end() or self.invalid_position()
        
        def invalid_position(self):
            return self.x > self.n

In [95]:
qn = question_1()
qn.run('input_question_1')

sum: 87127231192 is invalid
min: 4050144999
max: 14049945000
Sum exceeds maximum sum possible using this grid!
Try a value within [4050144999,14049945000]


KeyboardInterrupt: 